<a href="https://colab.research.google.com/github/chandini2595/Exploratory_Data_Analysis/blob/main/Apache_Beam_Features.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Install Apache Beam

In [ ]:
!pip install apache-beam

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.7/89.7 kB 4.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 621.9 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 152.0/152.0 kB 10.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.5/43.5 kB 2.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 173.5/173.5 kB 10.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 44.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.8/193.8 kB 14.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 304.6/304.6 kB 19.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.7/184.7 kB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━

#Basic Apache Beam Pipeline for Transforming and Printing Data

In [ ]:
import apache_beam as beam
from apache_beam.options.pipeline_options import PipelineOptions

# Define pipeline options
options = PipelineOptions()

# Create a pipeline
with beam.Pipeline(options=options) as p:
    (p
     | 'Read Input' >> beam.Create(['Hello', 'Apache', 'Beam', 'Streaming'])
     | 'Convert to Uppercase' >> beam.Map(lambda x: x.upper())
     | 'Print Results' >> beam.Map(print)
    )

HELLO
APACHE
BEAM
STREAMING


#Composite Transform

In [ ]:
class ProcessText(beam.PTransform):
    def expand(self, pcoll):
        return (pcoll
                | 'Convert to Uppercase' >> beam.Map(lambda x: x.upper())
                | 'Add Suffix' >> beam.Map(lambda x: f"{x} - processed"))

with beam.Pipeline(options=options) as p:
    (p
     | 'Read Input' >> beam.Create(['apple', 'banana', 'cherry'])
     | 'Process Text' >> ProcessText()
     | 'Print Results' >> beam.Map(print)
    )

APPLE - processed
BANANA - processed
CHERRY - processed


#Pipeline IO

In [ ]:
with beam.Pipeline(options=options) as p:
    (p
     | 'Read File' >> beam.io.ReadFromText('/content/sample_data/input.txt')
     | 'Process Text' >> beam.Map(lambda x: x.upper())
     | 'Write Results' >> beam.io.WriteToText('/content/sample_data/output.txt')
    )

#Triggers

In [ ]:
import time
import apache_beam as beam
from apache_beam.transforms.window import FixedWindows
from apache_beam.transforms.trigger import AfterProcessingTime, AccumulationMode
from apache_beam.options.pipeline_options import PipelineOptions

# Pipeline options
options = PipelineOptions()

# Simulate streaming data using a generator function
def generate_streaming_data():
    for i in range(10):
        yield f"event_{i}"
        time.sleep(1)  # Simulates data arriving every second

# Apache Beam pipeline
with beam.Pipeline(options=options) as p:
    (p
     | 'Create Stream' >> beam.Create(generate_streaming_data())
     | 'Window' >> beam.WindowInto(
            FixedWindows(5),
            trigger=AfterProcessingTime(3),
            accumulation_mode=AccumulationMode.DISCARDING  # Specify accumulation mode
        )
     | 'Print Results' >> beam.Map(print)
    )

event_0
event_1
event_2
event_3
event_4
event_5
event_6
event_7
event_8
event_9


#Windowing

In [ ]:
import time
import apache_beam as beam
from apache_beam.transforms.window import FixedWindows
from apache_beam.options.pipeline_options import PipelineOptions

# Pipeline options
options = PipelineOptions()

# Simulate streaming data using a generator function
def generate_streaming_data():
    for i in range(10):
        yield f"event_{i}"
        time.sleep(1)  # Simulates data arriving every second

# Apache Beam pipeline with Windowing
with beam.Pipeline(options=options) as p:
    (p
     | 'Create Stream' >> beam.Create(generate_streaming_data())  # Simulating streaming data
     | 'Window into Fixed Intervals' >> beam.WindowInto(FixedWindows(5))  # Window data into 5-second chunks
     | 'Print Windowed Results' >> beam.Map(print)  # Print results for each window
    )

event_0
event_1
event_2
event_3
event_4
event_5
event_6
event_7
event_8
event_9


#ParDo

In [ ]:
class ProcessElement(beam.DoFn):
    def process(self, element):
        yield element + " processed"

with beam.Pipeline(options=options) as p:
    (p
     | 'Read Input' >> beam.Create(['element1', 'element2', 'element3'])
     | 'Apply ParDo' >> beam.ParDo(ProcessElement())
     | 'Print Results' >> beam.Map(print)
    )

element1 processed
element2 processed
element3 processed
